In [1]:
import pandas as pd
import sys
import csv
from statistics import median

In [2]:
project_ytest_lib = {}

In [3]:
def get_delay_from_ci(ci, y_test, batch_size):
    
    
    y_test = output_values(y_test)
    
    sbs_list = []
    delay_list = []
    missed = []
    b = batch_size - 1
    
    i = 0

    while i < len(ci):
        sbs_delay = 0
        batch_delay = 0

        if ci[i] == 0:
            if y_test[i] == 0:
                sbs_list.append(0)

            while len(missed) > 0:
                ind = missed.pop()
                sbs_delay += i - ind
                sbs_list.append(i - ind)
            
            batch_delay += b
            b -= 1
            if b == -1:
                b = batch_size - 1

        if ci[i] == 1:
            if y_test[i] == 0:
                missed.append(i)
        
        delay_list.append(sbs_delay + batch_delay)
        i += 1
    while len(missed) > 0:
        ind = missed.pop()
        delay_list.append(i - ind)
        sbs_list.append(i - ind)
    
    return delay_list, sbs_list
    

In [4]:
def output_values(Y_data):
    Y_t = []
    for e in Y_data:
        if e == 'passed':
            Y_t.append(1)
        else:
            Y_t.append(0) 
    return Y_t

In [5]:
def get_best_confidence(data):
    
    #typecasting
    data['builds_reqd'] = pd.to_numeric(data['builds_reqd'], downcast="float", errors='coerce')   
    
    #setting parameters
    project_list = set(data['project'].tolist())
    batching_alg = set(data['algorithm'].tolist())
    batchsizes = [1,2,4,8,16]
    
    #result file
    result_file = open('best_confidence.csv', 'w')
    res_headers = ['project', 'method', 'algorithm', 'batch_size', 'builds_reqd', 'median_delays', 'delay_list', 'testall_size', 'confidence']
    res_writer = csv.writer(result_file)
    res_writer.writerow(res_headers)
    
    
    #get best confidences

    for p in project_list:
        
        p_name = p + '.csv'
        alg_scores = {}
        p_data = data[data['project'] == p]
        
        y_test = pd.read_csv('../../data/exact_data/' + p_name)['tr_status'].tolist()
        y_test = output_values(y_test)
        
        for alg in batching_alg:

            alg_data = p_data[p_data['algorithm'] == alg]

            for b in batchsizes:

                if alg == 'BATCH4':
                    if b != 4:
                        continue

                if alg == 'BATCHSTOP4':
                    if b < 4:
                        continue
                        

                batch_data = alg_data[ alg_data['batch_size'] == b]
                
                total = batch_data['testall_size'].tolist()[0]
                reqd_list = batch_data['builds_reqd'].tolist()
                median_delays = batch_data['median_delay'].tolist()
                delay_lists = batch_data['delay_list'].tolist()
                
                min_distance = sys.maxsize

                best = [p_name, 'ssr', alg, b, 0, 0, [], 0, 0]
                best_i = 0

                for i in range(len(reqd_list)):
                    distance = ((reqd_list[i]**2)+(median_delays[i])**2)**0.5
                    if distance < min_distance:
                        best[4] = reqd_list[i]
                        best[5] = median_delays[i]
                        best[6] = delay_lists[i]
                        best[7] = total
                        best[8] = i+2
                        best_i = i
                        min_distance = distance

                res_writer.writerow(best)
    

In [8]:
data = df.DataFrame()
projects = ['heroku', 'open-build-service', 'cloud_controller_ng', 'rubinius', 'jruby', 'metasploit-framework', 'loomio', 'orbeon-forms', 'cloudify', 'geoserver', 'opal', 'sufia', 'sonarqube', 'rails', 'fog', 'puppet', 'concerto', 'vagrant', 'gradle', 'graylog2-server']
for p in projects:
    file = p + '_confidences.csv'
    p_df = pd.read_csv(file)
    data = data.append(pdf)
    


    Unnamed: 0               project    algorithm  batch_size  confidence  \
0            0                 jruby  BATCHBISECT           4          20   
1            1                 jruby       BATCH4           4          20   
2            2                 jruby   BATCHSTOP4           4          20   
3            3                loomio  BATCHBISECT           4          20   
4            4                loomio       BATCH4           4          20   
5            5                loomio   BATCHSTOP4           4          20   
6            6    open-build-service  BATCHBISECT           4          20   
7            7    open-build-service       BATCH4           4          20   
8            8    open-build-service   BATCHSTOP4           4          20   
9            9                 sufia  BATCHBISECT           4          20   
10          10                 sufia       BATCH4           4          20   
11          11                 sufia   BATCHSTOP4           4          20   

In [9]:
get_best_confidence(data)

IndexError: list index out of range

92

In [24]:
result_file = open('sbs_delay_lists.csv', 'w')
res_headers = ['project', 'method', 'algorithm', 'batch_size', 'median_sbs_delays', 'median_batch_delays', 'confidence']
res_writer = csv.writer(result_file)
res_writer.writerow(res_headers)

86

In [25]:
lines = []

In [26]:
for p in project_list:
    p_name = p + '.csv'
    alg_scores = {}

    p_data = data[data['project'] == p]
    #print(p_data)
    for alg in batching_alg:

        alg_data = p_data[p_data['algorithm'] == alg]
        #print(alg_data)

        for b in batchsizes:

            if alg == 'BATCH4':
                if b != 4:
                    continue

            if alg == 'BATCHSTOP4':
                if b < 4:
                    continue

            batch_data = alg_data[ alg_data['batch_size'] == b]

            reqd_list = batch_data['project_reqd_builds'].tolist()
            delay_list = batch_data['total_sbs_delay'].tolist()
            batch_delay_list = batch_data['total_batch_delay'].tolist()
            ssr_delays = batch_data['project_delays'].tolist()
            min_distance = sys.maxsize

            best = [p_name, 'ssr', alg, b, 0, 0]
            best_i = 0

            for i in range(len(reqd_list)):
                distance = ((reqd_list[i]**2)+((delay_list[i]+batch_delay_list[i])**2))**0.5
                if distance < min_distance:
                    best[4] = ssr_delays[i]
                    #best[5] = median(ssr_delays[i])
                    best[5] = i+2
                    best_i = i
                    min_distance = distance

            lines.append(best)
            if alg == 'BATCH4':
                print(best)

['cloudify.csv', 'ssr', 'BATCH4', 4, '[]', 189]


In [27]:
ssr_delay = pd.DataFrame(lines, columns=['project', 'method', 'algorithm', 'batch_size', 'sbs_delays', 'confidence'])

In [28]:
ssr_delay.to_csv('sbs_delay_list.csv')